In [1]:
import scala.math.{abs, max, min}
import scala.util.Random

import $ivy.`org.plotly-scala::plotly-almond:0.8.1`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

import scala.math.{abs, max, min}

import scala.util.Random


import $ivy.$                                      


import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

In [2]:
case class Emotions(angry: Int, thankfulness: Int) {

  def update(payIn: Double, payOff: Double): Emotions = {
    if (payOff < 1.5 * payIn) {
      Emotions(between(angry + 1), between(thankfulness - 1))
    } else {
      Emotions(between(angry - 1), between(thankfulness + 1))
    }
  }

  private def between(value: Int, min: Int = 0, max: Int = 10): Int = {
    if (value < min) min
    else if (value > max) max
    else value
  }

  def emotionFactor: Double = {
    1.0 + (thankfulness - angry)/10
  }

  def +(other: Emotions): Emotions = {
    Emotions((this.angry + other.angry)/2, (this.thankfulness + other.thankfulness)/2)
  }
}

case class Personality(name: String, altruism: Double, egoism: Double, cooperating: Double) {

  lazy val features = Seq(altruism, egoism, cooperating)

  def ==(other: Personality): Boolean = {
    this.egoism == other.egoism && this.altruism == other.altruism && this.cooperating == other.cooperating
  }
}

object Personality {
  val altruistic: Personality = Personality("altruistic", 1.0, 0.0, 0.75)
  val egoistic: Personality = Personality("egoistic", 0.0, 1.0, 0.25)
  val impostor: Personality = Personality("impostor", 0.0, 1.0, 0.0)
  val ordinary: Personality = Personality("ordinary", 0.5, 0.5, 0.5)
  val erudite: Personality = Personality("erudite", 0.5, 0.75, 1.0)

  lazy val personalities = Seq(altruistic, egoistic, impostor, ordinary, erudite)

  def fromFeatures(altruism: Double, egoism: Double, cooperating: Double): Personality = {
    personalities.find(Personality("", altruism, egoism, cooperating) == _).getOrElse(throw new ClassNotFoundException("Personality not found"))
  }
}

case class Stat(personality: Personality, emotions: Emotions, amount: Double, payIn: Double, payOff: Double)

object Stats {

  def groupByPersonality(stats: List[Stat]): List[Stat] = {
    stats.groupBy(_.personality).map { case (personality, statList) =>
      Stat(
        personality,
        statList.map(_.emotions).reduce(_ + _),
        statList.map(_.amount).sum / statList.size,
        statList.map(_.payIn).sum / statList.size,
        statList.map(_.payOff).sum / statList.size
      )
    }.toList
  }

  def statsToString(stats: List[Stat]): String = {
    stats.mkString("\n")
  }
}

defined class Emotions
defined class Personality
defined object Personality
defined class Stat
defined object Stats

In [3]:
trait Player {

  val id: String
  val personality: Personality
  var emotions: Emotions = Emotions(5, 5)
  val community: Community
  val neighbourhood: Community = Community.empty

  var amount: Double = community.amount
  var lastPayoff: Double = 0
  var lastPayIn: Double = amount / 2

  def payout(payoff: Double): Unit = {
    lastPayoff = payoff
    amount += payoff
    emotions = emotions.update(lastPayIn, lastPayoff)
  }

  protected def b1: Double = emotions.emotionFactor * abs(personality.altruism * 0.75 + personality.cooperating * 0.25 - personality.egoism * 0.5) / 2

  protected def b2: Double = emotions.emotionFactor * abs(personality.altruism * 0.75 + personality.cooperating * 0.25 - personality.egoism * 1.0) / 2

  private def randomFactor: Double = (2 + (0.5 - Random.nextDouble())) / 2

  private def contribution: Double = {
    min(amount,  randomFactor * max(b1 * lastPayoff + b2 * (lastPayIn * community.size - lastPayoff) / (community.size - 1), 0))
  }

  def payIn: Double = {
    lastPayIn = contribution
    amount -= lastPayIn
    lastPayIn
  }

  override def toString: String = {
    getClass.getSimpleName + id + ":\t" + amount
  }
}

case class Altruist(id: String, community: Community) extends Player {
  override val personality: Personality = Personality.altruistic
}

case class Casual(id: String, community: Community) extends Player {

  override protected def b1: Double = Random.nextDouble()

  override protected def b2: Double = Random.nextDouble()

  override val personality: Personality = Personality.ordinary
}

case class Impostor(id: String, community: Community) extends Player {
  override val personality: Personality = Personality.impostor
}

case class Ordinary(id: String, community: Community) extends Player {

  override protected def b1: Double = 0.5

  override protected def b2: Double = 0.5

  override val personality: Personality = Personality.ordinary
}

case class Community(amount: Double, factor: Double) {

  private var players: List[Player] = Nil
  var statistics:  Map[Int, List[Stat]] = Map.empty

  def size: Int = players.size

  private def payIns(): Double = {
    players.map(_.payIn).sum
  }

  private def payOuts(pot: Double): Unit = {
    val payOff = pot * factor / size
    players.foreach(_.payout(payOff))
  }

  private def updateStatistics(roundIndex: Int): Unit = {
    statistics += (roundIndex -> players.map(player => Stat(player.personality, player.emotions, player.amount, player.lastPayIn, player.lastPayoff)))
  }

  def round(roundIndex: Int): List[Stat] = {
    val pot = payIns()
    payOuts(pot)
    updateStatistics(roundIndex)
    statistics(roundIndex)
  }

  def play(rounds: Int): Unit = {
    (1 to rounds).toList.foreach(idx => round(idx))
  }

  def withCasual(count: Int): Community = {
    addPlayers(count, Casual(_, this))
    this
  }

  def withAltruist(count: Int): Community = {
    addPlayers(count, Altruist(_, this))
    this
  }

  def withImpostor(count: Int): Community = {
    addPlayers(count, Impostor(_, this))
    this
  }

  def withOrdinary(count: Int): Community = {
    addPlayers(count, Ordinary(_, this))
    this
  }

  private def addPlayers(count: Int, playerCreation: String => Player): Unit = {
    players ++= (1 to count).toList.map(id => playerCreation(id.toString))
  }

  def getStatistic: List[(Int, List[Stat])] = statistics.toList.sortBy(_._1)(Ordering.Int)
}

object Community {
  val empty: Community = new Community(0, 0)
}

defined trait Player
defined class Altruist
defined class Casual
defined class Impostor
defined class Ordinary
defined class Community
defined object Community

In [4]:
val manhattan = Community(10.0, 1.5)
    .withAltruist(1)
    .withImpostor(1)
    .withOrdinary(1)

manhattan.play(50)



manhattan: Community = Community(amount = 10.0, factor = 1.5)

In [5]:
  val personalityAmount = manhattan.getStatistic.flatMap {
    case (round, stats) => stats.map(stat => (round, stat.personality.name, stat.amount))
  }.groupBy(_._2).mapValues(stats => stats.map(s => (s._1, s._3)).unzip).toMap

val personalityPayIn = manhattan.getStatistic.flatMap {
    case (round, stats) => stats.map(stat => (round, stat.personality.name, stat.payIn))
  }.groupBy(_._2).mapValues(stats => stats.map(s => (s._1, s._3)).unzip).toMap

personalityAmount: Map[String, (List[Int], List[Double])] = Map(
  "impostor" -> (
    List(
      1,
      2,
      3,
      4,
      5,
      6,
      7,
      8,
      9,
      10,
      11,
      12,
      13,
      14,
      15,
      16,
      17,
      18,
      19,
      20,
      21,
      22,
      23,
      24,
      25,
      26,
      27,
      28,
      29,
      30,
      31,
      32,
      33,
      34,
      35,
      36,
...
personalityPayIn: Map[String, (List[Int], List[Double])] = Map(
  "impostor" -> (
    List(
      1,
      2,
      3,
      4,
      5,
      6,
      7,
      8,
      9,
      10,
      11,
      12,
      13,
      14,
      15,
      16,
      17,
      18,
      19,
      20,
      21,
      22,
      23,
      24,
      25,
      26,
      27,
      28,
      29,
      30,
      31,
      32,
      33,
      34,
      35,
      36,
...

In [6]:
def data_scatter(data: Map[String, (List[Int], List[Double])]): Seq[Scatter] = {
    data.map { 
        case (key, (x, y)) => Scatter(x, y, name = key) 
    }.toSeq
}

defined function data_scatter

In [8]:
data_scatter(personalityAmount).plot()

: 

In [ ]:
plot_data(personalityPayIn)

In [ ]:
val (x, y) = Seq(
  "Banana" -> 10,
  "Apple" -> 8,
  "Grapefruit" -> 5
).unzip

Bar(x, y).plot()